In [ ]:
import numpy as np
import math
import json
import glob
from functools import reduce

In [ ]:
def load_results(dataset, model, metric, n_sample, seed):
    result_dir = f'../results/{dataset}/{model}_{seed}/{metric}_{n_sample}'
      
    result_fs = glob.glob(f'{result_dir}/*.json')
    results = []
    for file in result_fs:
        with open(file, 'r') as f:
            data = json.load(f)
            print(file, len(data))
        results.extend(data)
        
    return results

In [ ]:
metric = 'EMD'
dataset = 'Tiny-ImageNet'
arch = 'resnet50'
n_sample = 10000
seed = '1'
logits_results = load_results(dataset, arch, metric, n_sample, seed)

In [ ]:
len(logits_results)

In [ ]:
import altair as alt
import pandas as pd

In [ ]:
corr = alt.Chart(alt.Data(values=logits_results)).mark_point(size=80).encode(
    x=alt.X('metric:Q', title=metric),
    y=alt.X('error:Q', title='Test Error'),
    color=alt.Color('corruption:N', scale=alt.Scale(scheme='category20')),
    shape='corruption level:O'
).properties(
    width=350,
    height=350
)

line = pd.DataFrame({'metric': [0, 1], 'error': [0, 1]})
line_plot = alt.Chart(line).mark_line(color='black', strokeDash=[5, 8]).encode(
    x='metric',
    y='error',
)

if metric in ['Entropy', 'ProjNorm-ref']:
    plt = corr

else:
    plt = corr + line_plot

plt.configure_axis(
    labelFontSize=20,
    titleFontSize=20
)


In [ ]:
def polyfit(x, y, degree=1):
    results = {}

    coeffs = np.polyfit(x, y, degree)

    results['polynomial'] = coeffs.tolist()

    p = np.poly1d(coeffs)

    yhat = p(x)                
    ybar = np.sum(y)/len(y)          
    ssreg = np.sum((yhat - ybar)**2)   
    sstot = np.sum((y - ybar)**2)    
    results['determination'] = ssreg / sstot

    return results

In [ ]:
from scipy.stats import spearmanr
import math

In [ ]:
d = [i['metric'] for i in logits_results]
e = [i['error'] for i in logits_results]
print(f'{metric} r2:', polyfit(d, e)['determination'])

coef, p = spearmanr(d, e)
print(f'{metric} rho:', coef)

print(f'{metric} slope:', polyfit(d, e)['polynomial'][0])
print(f'{metric} bias:', polyfit(d, e)['polynomial'][1])

In [ ]:
yhat = np.array(d)
y = np.array(e)

print('MAE:', np.abs(yhat - y).mean() * 100)